In [29]:
from active_learning.learner.standard import Learner, get_classifier
from active_learning.weak_supervision.SelfTraining import SelfTraining
import argparse
import numpy as np
import copy
import pandas as pd
import math
import random
from sklearn.metrics import accuracy_score, f1_score
from timeit import default_timer as timer
from typing import List, Dict
from active_learning.config import get_active_config
from active_learning.dataStorage import DataStorage
from active_learning.datasets import load_synthetic
from active_learning.logger import init_logger
from active_learning.merge_weak_supervision_label_strategies.MajorityVoteLabelMergeStrategy import (
    MajorityVoteLabelMergeStrategy,
)
from collections import Counter

from active_learning.weak_supervision import SyntheticLabelingFunctions
from active_learning.weak_supervision.BaseWeakSupervision import BaseWeakSupervision

import seaborn as sns
import matplotlib.pyplot as plt
#sns.set_theme(style="whitegrid")


font_size = 8

tex_fonts = {
    # Use LaTeX to write all text
    # "text.usetex": True,
    "text.usetex": False,
    "font.family": ["Times New Roman", 'sans-serif'],
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": font_size,
    "font.size": font_size,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": font_size,
    "xtick.labelsize": font_size,
    "ytick.labelsize": font_size,
    "xtick.bottom": True,
    "figure.autolayout": True,
}

sns.set_style("whitegrid")
sns.set_context("paper")
plt.rcParams.update(tex_fonts)  # type: ignore


# https://jwalton.info/Embed-Publication-Matplotlib-Latex/
def set_matplotlib_size(width, fraction=1):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float
            Document textwidth or columnwidth in pts
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy

    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure (in pts)
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5 ** 0.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim


#width = 505.89
width = 1500


df = pd.read_csv("new.csv")
df['AMOUNT_OF_LFS'] = np.ceil(df['AMOUNT_OF_LFS'])
df.iloc[0].to_dict()

{'AMOUNT_OF_LFS': 2.0,
 'DATASET': 'HEART',
 'DATASET_RANDOM_GENERATION_SEED': 28296.0,
 'FRACTION_OF_INITIALLY_LABELLED_SAMPLES': 0.3742707957561203,
 'FRACTION_OF_LASTLY_AL_LABELLED_SAMPLES': 0.3706421470668909,
 'JOB_ID': 75.0,
 'LF_RANDOM_SEED': 238068.0,
 'MERGE_WS_SAMPLES_STRATEGY': 'MajorityVoteLabelMergeStrategy',
 'acc_al_and_al': 0.3708609271523179,
 'acc_al_and_ws_and_al': 0.8874172185430463,
 'acc_initial': 0.3973509933774834,
 'acc_ws': 0.9867549668874172,
 'al_sampling_strategy': 'UncertaintyMaxMargin_no_ws',
 'amount_of_initial_al_samples': 57.0,
 'amount_of_lastly_al_samples': 35.0,
 'class_sep': '?',
 'f1_al_and_al': 0.5410628019323671,
 'f1_al_and_ws_and_al': 0.9403508771929824,
 'f1_initial': 0.5687203791469194,
 'f1_ws': 0.9933333333333334,
 'flip_y': '?',
 'hypercube': '?',
 'n_classes': 2.0,
 'n_clusters_per_class': '?',
 'n_features': 14.0,
 'n_informative': 14.0,
 'n_redundant': 0.0,
 'n_repeated': 0.0,
 'n_samples': 303.0,
 'random_state': 28296.0,
 'scale': '?

Desirable Output: "exps with al_ws_al > ws_al": 
    lf_classifiers_knn:     80% vs 40% (all)
    amount_of_lf_features:  4.5 vs 7.0
    Offene Frage: macht es Sinn den Vergleich gegen ALL zu machen, oder lieber vergleich gegen "not davor"

In [39]:
def comparison_table(df, selection):
    print(len(df))
    filtered_df = df[selection]
    not_filtered_df = df[~selection]

    skip_keys = ['']
    for key in df.keys():
        if key in skip_keys:
            continue
        error_area = 1.96 * filtered_df.std() / math.sqrt(
            filtered_df.count()
        )
        not_error_area = 1.96 * not_filtered_df.std() / math.sqrt(
            not_filtered_df.count()
        )
        print("{:560}: {:>6.2%} ±{:>4.2%} vs {} ±{:>4.2%}".format(key, len(filtered_df), error_area, len(not_filtered_df), not_error_area) )

    '''
    mean = selection.mean()
    error_area = 1.96 * selection.std() / math.sqrt(
            selection.count()
        )
    results.append([key, mean, error_area])
    print(TITLE)
    print("{:>60} {:>6} {:>4}".format("", "Mean", ""))
    for result in results:
        print("{:<50} {:>6.2%} ±{:>4.2%}".format(result[0], float(result[1]), float(result[2])))
    '''
    
comparison_table(df, df['AMOUNT_OF_LF_FEATURESSS_4'] >0)

56034


TypeError: cannot convert the series to <class 'float'>

In [ ]:


# group by stuff
for group_by in ['DATASET', 'MERGE_WS_SAMPLES_STRATEGY']:
    print()
    print("#"*80)
    print(group_by)
    print("#"*80)
    for group_by_value in df[group_by].unique():
        print()
        print(group_by_value, ": ", len(df[df[group_by] == group_by_value]))
        for metric in ['acc', 'f1']:
            COMPARE_KEYS = [metric+"_initial", metric +"_ws"]
            for AL_SAMPLING_STRATEGY in ["UncertaintyMaxMargin_no_ws",
                    "UncertaintyMaxMargin_with_ws",
                    "Random",
                    "CoveredByLeastAmountOfLf",
                    "ClassificationIsMostWrong",
                    "GreatestDisagreement"]:
                    COMPARE_KEYS.append(metric + '_ws_and_al_'+AL_SAMPLING_STRATEGY)
                    COMPARE_KEYS.append(metric + '_al_and_al_'+AL_SAMPLING_STRATEGY)
            #comparison_table(df, COMPARE_KEYS=COMPARE_KEYS, TITLE="General Improvement after AL + WS " + metric)
            answer_questions(df[df[group_by] == group_by_value], COMPARE_KEYS=COMPARE_KEYS, metric=metric)